# Advanced DSPy Patterns

This notebook covers advanced DSPy patterns and techniques:
1. Bootstrapped few-shot learning
2. Self-improving modules
3. Optimization techniques
4. Custom metrics and evaluation

In [ ]:
import sys
sys.path.append('../')

from src.config import setup_dspy
from src.optimization_patterns import SelfImprovingModule, OptimizedChainOfThought, AdaptivePrompting

# Setup DSPy
lm = setup_dspy()

## 1. Self-improving Module Example
This example shows how to create a module that learns from its mistakes

In [ ]:
improver = SelfImprovingModule()

# First prediction without learning
result1 = improver("What causes earthquakes?", 
                  actual="Earthquakes are primarily caused by tectonic plate movements.")
print("Initial prediction:", result1['prediction'])

# Second prediction after learning
result2 = improver("What causes volcanoes?", 
                  actual="Volcanoes occur when magma reaches the Earth's surface through weak spots in the crust.")
print("\nImproved prediction:", result2['prediction'])
if 'improvement' in result2:
    print("Improvement strategy:", result2['improvement'])

## 2. Optimized Chain-of-Thought
Demonstrates how to optimize reasoning chains using bootstrapped examples

In [ ]:
# Create example data
examples = [
    {"input": "What is 25 + 17?", "output": "42"},
    {"input": "What is 13 × 4?", "output": "52"},
    {"input": "What is 100 ÷ 5?", "output": "20"}
]

# Create and compile the optimized module
cot = OptimizedChainOfThought()
cot.compile_with_examples(examples)

# Test the optimized module
result = cot("What is 36 + 44?")
print(f"Question: What is 36 + 44?")
print(f"Reasoning: {result['reasoning']}")
print(f"Answer: {result['conclusion']}")

## 3. Adaptive Prompting
Shows how to adapt prompting strategy based on input complexity

In [ ]:
adaptive = AdaptivePrompting()

# Test with simple input
simple_input = "What is the capital of France?"
simple_result = adaptive(simple_input)
print(f"Simple input ({simple_result['method']}):\n{simple_result['output']}\n")

# Test with complex input
complex_input = "Compare and contrast the economic systems of capitalism and socialism"
complex_result = adaptive(complex_input)
print(f"Complex input ({complex_result['method']}):\n{complex_result['output']}")
if 'reasoning' in complex_result:
    print(f"\nReasoning:\n{complex_result['reasoning']}")

# Advanced DSPy Patterns and Techniques

This notebook demonstrates advanced DSPy patterns including:
1. Signature chaining
2. Dynamic few-shot learning
3. Error handling and self-correction
4. Prompt optimization

In [ ]:
import sys
sys.path.append('../')

import dspy
from src.config import setup_dspy

# Setup DSPy
lm = setup_dspy()

## Signature Chaining Example
This example shows how to chain multiple signatures for complex tasks

In [ ]:
class ComplexReasoningChain(dspy.Module):
    def __init__(self):
        super().__init__()
        self.analyze = dspy.ChainOfThought("input -> key_points, implications")
        self.synthesize = dspy.ChainOfThought("key_points, implications -> conclusion")
        self.validate = dspy.ChainOfThought("conclusion, key_points -> is_valid, reasoning")
    
    def forward(self, input_text):
        # Step 1: Analysis
        analysis = self.analyze(input=input_text)
        
        # Step 2: Synthesis
        synthesis = self.synthesize(
            key_points=analysis.key_points,
            implications=analysis.implications
        )
        
        # Step 3: Validation
        validation = self.validate(
            conclusion=synthesis.conclusion,
            key_points=analysis.key_points
        )
        
        return {
            'conclusion': synthesis.conclusion,
            'valid': validation.is_valid,
            'reasoning': validation.reasoning
        }

# Test the complex reasoning chain
reasoner = ComplexReasoningChain()
input_text = "AI models are becoming increasingly powerful, but with this power comes concerns about safety and ethics."
result = reasoner(input_text)
print(f"Input: {input_text}")
print(f"Conclusion: {result['conclusion']}")
print(f"Valid: {result['valid']}")
print(f"Reasoning: {result['reasoning']}")

## Dynamic Few-Shot Learning
This example demonstrates how to dynamically generate and use few-shot examples

In [ ]:
class DynamicFewShotLearner(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_examples = dspy.ChainOfThought("task_description -> examples")
        self.solve = dspy.ChainOfThought("examples, new_input -> solution")
    
    def forward(self, task_description, new_input):
        # Generate relevant examples
        examples_gen = self.generate_examples(task_description=task_description)
        
        # Use examples to solve new input
        solution = self.solve(
            examples=examples_gen.examples,
            new_input=new_input
        )
        
        return solution.solution

# Test the dynamic few-shot learner
learner = DynamicFewShotLearner()
task = "Convert informal text to formal business language"
input_text = "Hey, just checking if you got my email about the project?"
result = learner(task, input_text)
print(f"Input: {input_text}")
print(f"Formal version: {result}")

## Prompt Optimization Example
This example shows how to use DSPy's optimization capabilities

In [ ]:
from dspy.teleprompt import BootstrapFewShot

class OptimizedModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.process = dspy.ChainOfThought("input -> output")
    
    def forward(self, input_text):
        result = self.process(input=input_text)
        return result.output

def create_example_dataset():
    examples = [
        {"input": "Summarize the benefits of exercise", "output": "Exercise improves health, mood, and longevity."},
        {"input": "Explain quantum computing", "output": "Quantum computing uses quantum mechanics for complex calculations."}
    ]
    return dspy.Example.from_list(examples)

# Create and optimize the module
module = OptimizedModule()
bootstrap = BootstrapFewShot(metric="exact_match")
trainset = create_example_dataset()

# Compile with optimization
optimized_module = bootstrap.compile(
    module,
    trainset=trainset,
    max_bootstrapped_demos=3
)

# Test the optimized module
result = optimized_module("Describe machine learning")
print(f"Result: {result}")